# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

## LLM's on Documents
- LLM's can only inspect a few thousand words at a time.

![Embedding](./images/embedding.png)
![Embedding](./images/vector_db.png)

In [1]:
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown
from langchain.embeddings import OpenAIEmbeddings

In [25]:
# %pip install docarray
# %pip install tiktoken

In [124]:
model = "qwen2:1.5b"
llm = Ollama(temperature = 0.0, model=model)
llm_replacement_model = Ollama(temperature=0, model=model)

file = 'outdoor_clothing_catalog.csv'
loader = CSVLoader(file_path=file)

In [125]:
from langchain.embeddings.base import Embeddings
import requests

class OllamaEmbeddings(Embeddings):
    def __init__(self, model_name, ollama_url='http://localhost:11434'):
        self.model_name = model_name
        self.ollama_url = ollama_url
    
    def embed_documents(self, texts):
        return [self._get_embedding(text) for text in texts]
    
    def embed_query(self, text):
        return self._get_embedding(text)
    
    def _get_embedding(self, text):
        response = requests.post(
            f'{self.ollama_url}/api/embeddings',
            json={
                "model": self.model_name,
                "prompt": text
            }
        )
        response.raise_for_status()
        return response.json()['embedding']  # Adjust if the key is different


In [88]:
# embedding = OpenAIEmbeddings(openai_api_key='your-openai-api-key')
embedding = OllamaEmbeddings(model_name="nomic-embed-text")

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,embedding=embedding
).from_loaders([loader])

In [126]:
query ="Please list all your shirts with sun protection in a table in markdown and summarize each one."

In [127]:
response = index.query(query, llm = llm_replacement_model)

In [128]:
display(Markdown(response))

| Shirt Name | Description | Materials Used |
| --- | --- | --- |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry. | 
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. | 
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. | 
| Women's Tropical Tee, Sleeveless | Our five-star sleeveless button-up shirt has a fit to flatter and SunSmart™ protection to block the sun’s harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Machine wash and dry. | 
| Women's Tropical Tee, Sleeveless | Our five-star sleeveless button-up shirt has a fit to flatter and SunSmart™ protection to block the sun’s harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Machine wash and dry. |

In [129]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'outdoor_clothing_catalog.csv', 'row': 0}, page_content=": 0\nUnnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [131]:
embed = embedding.embed_query("Hi my name is Rabin")
print(len(embed))
print(embed[:5])

768
[0.005068518221378326, -0.06807872653007507, -2.709683418273926, 0.6819742321968079, 0.5828462243080139]


In [113]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

In [132]:
query = "Please suggest a shirt with sunblocking"
docs = db.similarity_search(query)
print(len(docs))
print(docs[0])

4
page_content=': 255
Unnamed: 0: 255
name: Sun Shield Shirt by
description: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. 

Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.

Fabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.

Additional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.

Sun Protection That Won't Wear Off
Our high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun's harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.' metadata={'source': 'outdoor_clothing_catalog.csv', 'row': 255}


## Question Answering

In [133]:
retriever = db.as_retriever()
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [134]:
# openai response = lllm.call_as_llm("")
response = llm.generate(prompts=[f"{qdocs} Question: Please list all your shirts with sun protection in a table in markdown and summarize each one."]) 

In [135]:
print(response.generations[0][0].text)

| Shirt Name | Description |
| --- | --- |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. |
| Tropical Breeze Shirt | Beat the heat in this lightweight, breathable long-sleeve men’s UPF shirt, offering superior SunSmart™ protection from the sun’s harmful rays. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. |
| Tropical Breeze Shirt | Beat the heat in this lightweight, breathable long-sleeve men’s UPF shirt, offering superior SunSmart™ protection from the sun’s harmful rays. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective but

In [136]:
display(Markdown(response.generations[0][0].text))


| Shirt Name | Description |
| --- | --- |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. |
| Tropical Breeze Shirt | Beat the heat in this lightweight, breathable long-sleeve men’s UPF shirt, offering superior SunSmart™ protection from the sun’s harmful rays. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. |
| Tropical Breeze Shirt | Beat the heat in this lightweight, breathable long-sleeve men’s UPF shirt, offering superior SunSmart™ protection from the sun’s harmful rays. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. |

Table: 
| Shirts with Sun Protection |
| --- |
| Name | Description |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. |
| Tropical Breeze Shirt | Beat the heat in this lightweight, breathable long-sleeve men’s UPF shirt, offering superior SunSmart™ protection from the sun’s harmful rays. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women’s UV-protective button down shirt. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. |

Summary:
- Sun Shield Shirt by offers high-performance sun protection from harmful UV rays.
- Tropical Breeze Shirt provides lightweight, breathable long-sleeve men’s UPF shirt with superior SunSmart™ protection.
- Sunrise Tee Stay cool, comfortable and dry on the hottest days in our women’s UV-protective button down shirt. 
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry.

In [137]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

## Stuff method
- docs -> prompt -> LLM -> Final Answer

* Stuffing is the simplest method. You simply stuff all data into the prompt as context to pass to the language model.
* Pros: It makes a single call to the LLM. The LLM as access to tall the data at once.
* Cons: LLMs have a context length, and for large documents or many documents this will not work as it will result in a prompt larger than the context length.

### 3 additional methods
1. Map_reduce (used for summarization)
2. Refine
3. Map_rerank

![Stuff](./images/stuff_method.png)
![Stuff](./images/map_reduce.png)
![Stuff](./images/refine.png)
![Stuff](./images/map_rerank.png)

In [138]:
query =  "Please list all your shirts with sun protection in a table in markdown and summarize each one."
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [139]:
display(Markdown(response))

| Shirt Name | Description | Materials Used |
| --- | --- | --- |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. | 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. |
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly. |
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. |
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. |
| Sunrise Tee | Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly.

In [ ]:
response = index.query(query, llm=llm)
response

In [140]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding,
).from_loaders([loader])

In [141]:
query ="Please list all your shirts with sun protection in a table in markdown and summarize each one."
response = index.query(query, llm = llm_replacement_model)
display(Markdown(response))

| Shirt Name | Description | Materials Used |
| --- | --- | --- |
| Sun Shield Shirt by | Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry. | 
| Men's Plaid Tropic Shirt, Short-Sleeve | Our Ultracomfortable sun protection is rated to UPF 50+, helping you stay cool and dry. Originally designed for fishing, this lightest hot-weather shirt offers UPF 50+ coverage and is great for extended travel. SunSmart technology blocks 98% of the sun's harmful UV rays, while the high-performance fabric is wrinkle-free and quickly evaporates perspiration. Made with 52% polyester and 48% nylon, this shirt is machine washable and dryable. | 
| Men's Tropical Plaid Short-Sleeve Shirt | Our lightest hot-weather shirt is rated UPF 50+ for superior protection from the sun's UV rays. With a traditional fit that is relaxed through the chest, sleeve, and waist, this fabric is made of 100% polyester and is wrinkle-resistant. With front and back cape venting that lets in cool breezes and two front bellows pockets, this shirt is imported and provides the highest rated sun protection possible. | 
| Women's Tropical Tee, Sleeveless | Our five-star sleeveless button-up shirt has a fit to flatter and SunSmart™ protection to block the sun’s harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Machine wash and dry. | 
| Women's Tropical Tee, Sleeveless | Our five-star sleeveless button-up shirt has a fit to flatter and SunSmart™ protection to block the sun’s harmful UV rays. Size & Fit: Slightly Fitted: Softly shapes the body. Falls at hip. Fabric & Care: Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. Machine wash and dry. |